In [2]:
import xgboost as xg
# import dask.dataframe as dd
import polars as pl
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

In [3]:
def test_rankings(pred, order):
    for srch in pred.srch_id.unique():
        srch_pred = pred[pred.srch_id == srch]
        srch_order = order[order.srch_id == srch]
        assert len(srch_pred) == len(srch_order)
        assert (srch_pred.prop_id == srch_order.prop_id).all()

In [4]:
data = pl.read_csv('../data/raw/training_set_VU_DM.csv')
ranking = pl.read_csv('../data/preprocessed/query_order.csv')

data.describe()

statistic,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
str,f64,str,f64,f64,str,str,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,f64
"""count""",4.958347e6,"""4958347""",4.958347e6,4.958347e6,"""4958347""","""4958347""",4.958347e6,4.958347e6,4.958347e6,4.950983e6,4.958347e6,4.958347e6,"""4958347""",4.958347e6,4.958347e6,4.958347e6,4.958347e6,4.958347e6,4.958347e6,4.958347e6,4.958347e6,4.958347e6,4.958347e6,4.958347e6,"""4958347""","""4958347""",4.958347e6,"""4958347""","""4958347""","""4958347""","""4958347""","""4958347""","""4958347""","""4958347""","""4958347""","""4958347""","""4958347""","""4958347""","""4958347""","""4958347""","""4958347""","""4958347""","""4958347""","""4958347""","""4958347""","""4958347""","""4958347""","""4958347""","""4958347""","""4958347""","""4958347""",4.958347e6,"""4958347""",4.958347e6
"""null_count""",0.0,"""0""",0.0,0.0,"""0""","""0""",0.0,0.0,0.0,7364.0,0.0,0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0""","""0""",0.0,"""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""",0.0,"""0""",0.0
"""mean""",166366.561096,null,9.953133,175.340453,null,null,173.973897,70079.179496,3.180525,3.777777,0.634699,2.872589,null,4.317913,16.856236,254.20959,0.21562,14042.630393,2.385427,37.474165,1.972637,0.350492,1.110525,0.502213,null,null,0.2959,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.044749,null,0.027911
"""std""",96112.230102,null,7.64689,65.916249,null,null,68.345248,40609.920378,1.051024,1.050329,0.481514,1.531011,null,1.834869,10.425655,16001.237061,0.411252,8111.843351,2.053243,51.993411,0.857063,0.732569,0.417229,0.499995,null,null,0.456446,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.206751,null,0.164716
"""min""",1.0,"""2012-11-01 00:08:29""",1.0,1.0,"""1.41""","""0.0""",1.0,1.0,0.0,0.0,0.0,0.0,"""0.0""",0.0,1.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,"""-10.0001""","""0.01""",0.0,"""-1""","""-1""","""10""","""-1""","""-1""","""10""","""-1""","""-1""","""10""","""-1""","""-1""","""10""","""-1""","""-1""","""10""","""-1""","""-1""","""10""","""-1""","""-1""","""10""","""-1""","""-1""","""10""",0.0,"""0.0""",0.0
"""25%""",82936.0,null,5.0,100.0,null,null,100.0,35010.0,3.0,3.5,0.0,1.79,null,4.45,8.0,85.0,0.0,7101.0,1.0,4.0,2.0,0.0,1.0,0.0,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,0.0
"""50%""",166507.0,null,5.0,219.0,null,null,219.0,69638.0,3.0,4.0,1.0,2.77,null,4.91,16.0,122.0,0.0,13541.0,2.0,17.0,2.0,0.0,1.0,1.0,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,0.0
"""75%""",249724.0,null,14.0,219.0,null,null,219.0,105168.0,4.0,4.5,1.0,4.04,null,5.

In [5]:
ranking.describe()

statistic,srch_id,prop_id
str,f64,f64
"""count""",4.958347e6,4.958347e6
"""null_count""",0.0,0.0
"""mean""",166366.561096,70079.179496
"""std""",96112.230102,40609.920378
"""min""",1.0,1.0
"""25%""",82936.0,35010.0
"""50%""",166507.0,69638.0
"""75%""",249724.0,105168.0
"""max""",332785.0,140821.0


In [6]:
print(data.shape)
print(ranking.shape)

(4958347, 54)
(4958347, 2)


In [7]:
data = pl.read_csv('../data/raw/training_set_VU_DM.csv')
ranking = pl.read_csv('../data/preprocessed/query_order.csv')

# Read the data and ranking
data = pl.read_csv('../data/raw/training_set_VU_DM.csv')
ranking = pl.read_csv('../data/preprocessed/query_order.csv')

# Convert polars DataFrames to pandas DataFrames
data_pd = data.to_pandas()
ranking_pd = ranking.to_pandas()


# Convert object columns to appropriate data types
object_columns = data_pd.select_dtypes(include=['object']).columns
data_pd[object_columns] = data_pd[object_columns].apply(pd.to_numeric, errors='coerce')

# Split the data into features (X) and target (y)
X = data_pd.drop(['srch_id', 'prop_id'], axis=1)
y = ranking_pd['prop_id']


# Split the data into training and testing sets based on srch_id
srch_ids = data_pd['srch_id'].unique()
train_srch_ids, test_srch_ids = train_test_split(srch_ids, test_size=0.2, random_state=42)

# Create training and testing DataFrames
train_data = data_pd[data_pd['srch_id'].isin(train_srch_ids)]
test_data = data_pd[data_pd['srch_id'].isin(test_srch_ids)]

# Create training and testing ranking DataFrames
train_ranking = ranking_pd[ranking_pd['srch_id'].isin(train_srch_ids)]
test_ranking = ranking_pd[ranking_pd['srch_id'].isin(test_srch_ids)]

# Create XGBoost DMatrix objects for training and testing
train_dmatrix = xg.DMatrix(train_data.drop(['srch_id', 'prop_id'], axis=1), label=train_ranking['prop_id'])
test_dmatrix = xg.DMatrix(test_data.drop(['srch_id', 'prop_id'], axis=1), label=test_ranking['prop_id'])

# Set XGBoost parameters
params = {
    'objective': 'rank:pairwise',
    'eval_metric': 'ndcg',
    'learning_rate': 0.1,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'seed': 42
}

# Train the XGBoost ranking model
model = xg.train(params, train_dmatrix, num_boost_round=100)



In [13]:
from sklearn.metrics import ndcg_score
# Make predictions on the test set
test_pred = model.predict(test_dmatrix)

# Evaluate the model
ndcg_scores = []
for srch_id in test_srch_ids:
    test_pred_query = test_pred[test_data['srch_id'] == srch_id]
    test_ranking_query = test_ranking[test_ranking['srch_id'] == srch_id]['prop_id']
    ndcg = ndcg_score(test_ranking_query.values.reshape(1, -1), test_pred_query.reshape(1, -1), k=5)
    ndcg_scores.append(ndcg)

# Calculate the mean NDCG score
mean_ndcg = sum(ndcg_scores) / len(ndcg_scores)
print(f"Mean NDCG: {mean_ndcg:.4f}")

Mean NDCG: 0.5807


In [14]:
# save the model

model.save_model('model.json')